# About this notebook

Implemented by Rodrigo Costa (rccosta@stanford.edu), February, 2022.
This notebook implements the algorithms in the Hazus Building Inventory Technical Manual and can be used to generate **synthetic** building inventories for communities. It requires two files:


1.   A text file containing the location of each building (Example_Inventory.txt)
2.   A shapefile containing the borders of the Census Tracts in the community (Alameda_census_blockgroups_mini.geojson)
3.   A file containing minimum information about the demographics in each Census Tract (Alameda_Block_Group_IR.txt)

Note: the outputs from this file **must not** be used to draw conclusions about any building in Alameda (CA) the community used as example in this notebook.




---



---



# Imports

In [20]:
# Mount YOUR google drive. You'll need to "Add shortcut to Drive" for our shared folder for it to show up here.
# Use the URL shown below in the output to authorize this Colab session to access you GDrive
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

# Modify this according to the path in your computer
data_dir = '/content/drive/MyDrive/SURI/RAAbIT/HazusBuildingInventory/' # <-- change this to reflect the pathing in your machine

Mounted at /content/drive/


In [21]:
# Import needed packages
! pip install geopandas
! pip install geopy
! pip install -U plotly

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geopy.distance
from shapely.geometry import Point, Polygon
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy import stats as sts
import plotly.express as px
from numpy.random import default_rng
from plotnine import *

rng = default_rng(13)

# Get hold of the tax assessor data

In [22]:
# Get hold of the tax assessor data
df_Alameda = pd.read_csv(data_dir +"Example_Inventory.txt", delim_whitespace=False, header=0)

df_Alameda.head(5)

,index,Latitude,Longitude,UseCategory,CensusBlock,Units
0,1,37.751725,-122.237448,Single-family Detached,60014282001,1
1,2,37.751599,-122.237493,Single-family Detached,60014282001,1
2,3,37.751596,-122.237625,Single-family Detached,60014282001,1
3,4,37.752485,-122.237371,Single-family Detached,60014282001,1
4,5,37.752420,-122.237493,Single-family Detached,60014282001,1


In [23]:
# Get hold of the census blocks
df_Blocks = gpd.read_file(data_dir+'Alameda_census_blockgroups_mini.geojson')
df_Alameda['CensusBlock'] = [-1] * len(df_Alameda.index)

# Loop over all buildings and check their census block
for i in range(len(df_Alameda.index)):
    for j in range(len(df_Blocks.index)):
        point = Point(df_Alameda.loc[i,'Longitude'],df_Alameda.loc[i,'Latitude'])
        check = point.within(df_Blocks.loc[j,'geometry'])

        if check == True:
            df_Alameda.loc[i,'CensusBlock'] = int(df_Blocks.loc[j,'GEOID10'])
            break

# Drop buildings for with missing information
df_Alameda = df_Alameda[df_Alameda['CensusBlock'] != -1].reset_index(drop=True)
df_Alameda = df_Alameda.dropna().reset_index(drop=True)

# Get updated number of buildings
nblds = df_Alameda.shape[0]

# Next, get the Income Ratio for each Census Tract which is used later to estimate building replacement costs
# Income Ratio = Median Household Income for Census tract / Median Area Household Income 
df_IR = pd.read_csv(data_dir +"Alameda_Block_Group_IR.txt", delim_whitespace=False, header=0)
df_Alameda['IncomeRatio'] = [df_IR.loc[list(df_IR['FIPS']).index(int(df_Alameda.loc[i,'CensusBlock'])),'IR'] for i in range(len(df_Alameda.index))]

# Building attributes

## Year built

In [24]:
def setYearBuiltFromDistribution(s,p,n):
    # s: vector of states which may represent the decade the building was built, e.g., [1940,1950,1960]
    # p: vector of probabilities of being of each state - should be the same size as s 
    # n: number of buildings

    # normalize p
    p = [float(i)/sum(p) for i in p]
    return rng.choice(s, p=p, size=n, replace=True)

## Building code

In [25]:
# From Hazus Inventory Technical Manual Section XXX
def f_code(bld_age):
  bld_code = 'Post1940'
  if (bld_age < 1940):
    bld_code = 'Pre1940'
  elif (bld_age > 1980):
    bld_code = 'Post1980'
  
  return bld_code

## Construction class

In [26]:
# From Hazus Inventory Technical Manual Section XXX
def setConstructionClassFromHazus(input_df):
    # region: location in the US
    # ir: income ration

    s = ['Luxury','Custom','Average','Economic']
    p = []
    v = ['None'] * len(input_df.index)

    for i in range(len(input_df)):
        ir = input_df.loc[i,'IncomeRatio']

        if ir < 0.5:
            p = [0,0,0,1]

        elif 0.5 <= ir < 0.85: 
            p = [0,0,0.25,0.75]

        elif 0.85 <= ir < 1.25: 
            p = [0,0.25,0.75,0]

        elif 1.25 <= ir < 2.0: 
            p = [0,1,0,0]

        else: 
            p = [1,0,0,0]

        v[i] = rng.choice(s, p=p, size=1, replace=True)[0]

    return v

## Number to stories

In [27]:
def setStoriesFromHazus(region,input_df):
    # From Section 5.5.6 in the Hazus Inventory Technical Manual
    # region: Northeast, Midwest, South, or West
    # occupancy: RES1 for detached buildings, RES3A and RES3B for duplexes and triplexes
    # n: number of buildings

    # The number of stories is defined according to the Hazus Inventory Manual
    # Values from Table 5-4 Hazus Inventory manual
    s_stories = []
    p_stories = []
    v = [0] * len(input_df.index)
    
    for i in range(len(input_df.index)):

        occupancy = input_df.loc[i,'Occupancy']

        if occupancy == 'RES1':
            if region == 'Northeast':
                s_stories = [1, 2, 3]
                p_stories = [0.25,0.68,0.05] 

            elif region == 'Midwest':
                s_stories = [1, 2, 3]
                p_stories = [0.50,0.46,0.02]

            elif region == 'South':
                s_stories = [1, 2, 3]
                p_stories = [0.66,0.32,0.01]

            else: # West
                s_stories = [1, 2, 3]
                p_stories = [0.66,0.30,0.02]

            p_stories = [float(i)/sum(p_stories) for i in p_stories]
            v[i] = rng.choice(s_stories, p=p_stories, size=1, replace=True)[0]

        # RES3 building fall into two categories 1-2, and 3-4 stories
        else:
            if region == 'Northeast':
                s_stories = [1,3]
                p_stories = [0.29,0.26] 

            elif region == 'Midwest':
                s_stories = [1,3]
                p_stories = [0.49,0.29]

            elif region == 'South':
                s_stories = [1,3]
                p_stories = [0.66,0.21]

            else: # West
                s_stories = [1,3]
                p_stories = [0.58,0.25]

            p_stories = [float(i)/sum(p_stories) for i in p_stories]
            v[i] = int(rng.choice(s_stories, p=p_stories, size=1, replace=True)[0] + round(rng.random()))

    return v

## Basement

In [28]:
def setBasementFromHazus(region,n):
    # From Tables 5-3 and 5-6 in the Hazus Inventory Technical Manual
    # region: region in the United States
    # n: number of buildings
    s = [True, False]
    p = []

    if region == 'New England':
        p = [0.72,0.28]

    elif region == 'Mid Atlantic':
        p = [0.58,0.42]

    elif region == 'East North Central':
        p = [0.57,0.42]

    elif region == 'West North Central':
        p = [0.56,0.44]

    elif region == 'South Atlantic':
        p = [0.19,0.81]

    elif region == 'East South Central':
        p = [0.18,0.82]

    elif region == 'West South Central':
        p = [0.02,0.98]

    elif region == 'Mountain':
        p = [0.26,0.74]

    else: # West
        p = [0.08, 0.92]

    p = [float(i)/sum(p) for i in p]  
    return rng.choice(s, p=p, size=n, replace=True)

## Area

In [29]:
def setAreaFromHazus(region,df_input):
    # From Table 5-2 in the Hazus Inventory Technical Manual
    # region: census region
    # n: number of buildings
    # ir: income ratio
    # bsm: presence of basement, True or False
    # occ: occupancy
    # units: number of units in multi-unit buildings

    n = len(df_input.index)
    ir = df_input['IncomeRatio']
    bsm = df_input['Basement']
    occ = df_input['Occupancy']
    units = df_input['Units']
    area = [0] * n

    for i in range(n):
        if occ[i] == 'RES1':
            if region == 'New England':
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1300
                    else:
                        area[i] = 975

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1500
                    else:
                        area[i] = 1125

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1800
                    else:
                        area[i] = 1350

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 1900
                    else:
                        area[i] = 1425

                else:
                    if bsm[i] == False:
                        area[i] = 2200
                    else:
                        area[i] = 1650

            elif region == 'Middle Atlantic':
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1300
                    else:
                        area[i] = 975

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1500
                    else:
                        area[i] = 1125

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1700
                    else:
                        area[i] = 1275

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 1900
                    else:
                        area[i] = 1425

                else:
                    if bsm[i] == False:
                        area[i] = 2200
                    else:
                        area[i] = 1650

            elif region == 'East North Central':
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1300
                    else:
                        area[i] = 975

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1600
                    else:
                        area[i] = 1200

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1700
                    else:
                        area[i] = 1275

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 1800
                    else:
                        area[i] = 1350

                else:
                    if bsm[i] == False:
                        area[i] = 2500
                    else:
                        area[i] = 1875

            elif region == 'West North Central':
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1300
                    else:
                        area[i] = 975

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1500
                    else:
                        area[i] = 1125

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1800
                    else:
                        area[i] = 1350

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 1800
                    else:
                        area[i] = 1350

                else:
                    if bsm[i] == False:
                        area[i] = 2300
                    else:
                        area[i] = 1725

            elif region == 'South Atlantic':
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1400
                    else:
                        area[i] = 1050

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1600
                    else:
                        area[i] = 1200

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1700
                    else:
                        area[i] = 1275

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 2000
                    else:
                        area[i] = 1500

                else:
                    if bsm[i] == False:
                        area[i] = 2300
                    else:
                        area[i] = 1725

            if region == 'East South Central':
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1300
                    else:
                        area[i] = 975

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1400
                    else:
                        area[i] = 1050

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1700
                    else:
                        area[i] = 1275

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 1900
                    else:
                        area[i] = 1425

                else:
                    if bsm[i] == False:
                        area[i] = 2500
                    else:
                        area[i] = 1875

            if region == 'West South Central':
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1300
                    else:
                        area[i] = 975

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1700
                    else:
                        area[i] = 1275

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1800
                    else:
                        area[i] = 1350

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 1900
                    else:
                        area[i] = 1425

                else:
                    if bsm[i] == False:
                        area[i] = 2500
                    else:
                        area[i] = 1875

            elif region == 'Mountain':
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1200
                    else:
                        area[i] = 900

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1500
                    else:
                        area[i] = 1125

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1700
                    else:
                        area[i] = 1275

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 1800
                    else:
                        area[i] = 1350

                else:
                    if bsm[i] == False:
                        area[i] = 2600
                    else:
                        area[i] = 1950

            else: # West
                if ir[i] < 0.5:
                    if bsm[i] == False:
                        area[i] = 1300
                    else:
                        area[i] = 975

                elif 0.5 <= ir[i] < 0.85:
                    if bsm[i] == False:
                        area[i] = 1500
                    else:
                        area[i] = 1125

                elif 0.85 <= ir[i] < 1.25:
                    if bsm[i] == False:
                        area[i] = 1700
                    else:
                        area[i] = 1275

                elif 1.25 <= ir[i] < 2:
                    if bsm[i] == False:
                        area[i] = 1900
                    else:
                        area[i] = 1425

                else:
                    if bsm[i] == False:
                        area[i] = 2100
                    else:
                        area[i] = 1575
        elif occ[i] == 'RES3A' or occ[i] == 'RES3B':

            if region == 'Northeast':
                area[i] = 1191 * units[i]
            elif region == 'Midwest':
                area[i] = 1279 * units[i]
            elif region == 'South':
                area[i] = 945 * units[i]
            else: #West
                area[i] = 930 * units[i]

        else: #RES3C-RES3F
            if region == 'Northeast':
                area[i] = 849 * units[i]
            elif region == 'Midwest':
                area[i] = 787 * units[i]
            elif region == 'South':
                area[i] = 916 * units[i]
            else: #West
                area[i] = 811 * units[i]

    return area

## Replacement cost

In [30]:
def setReplacementCostFromHazus(input_df):
    # From Section 5.9.2 in the Hazus Inventory Technical Manual
    # Assumption
    bsm_finish = True
    
    v = [0] * len(input_df.index)

    for i in range(len(input_df.index)):

        bld_occ     = input_df.loc[i,'Occupancy']
        const_class = str(input_df.loc[i,'ConstructionClass'])
        bld_area    = input_df.loc[i,'Area']
        n_storeys   = input_df.loc[i,'Stories']
        bsm_flag    = input_df.loc[i,'Basement']

        bsm_area = 0.0
        if (bsm_flag == True):
            bsm_area = bld_area * 0.25

        
        # Adapted from Building Portfolio Manual Table 6.3
        bld_value = 0.0
        if (bld_occ == 'RES1'):
            if (const_class == 'Economic'):
                if n_storeys == 1:
                    bld_value = 97.61 * bld_area + 26.45 * bsm_area if bsm_finish == True else 9.55 * bsm_area
                elif n_storeys == 2:
                    bld_value = 104.04 * bld_area + 15.20 * bsm_area if bsm_finish == True else 6.30 * bsm_area
                elif n_storeys >= 3:
                    bld_value = 104.04 * bld_area + 15.20 * bsm_area if bsm_finish == True else 6.30 * bsm_area
                    
            elif (const_class == 'Average'):
                if n_storeys == 1:
                    bld_value = 116.66 * bld_area + 32.80 * bsm_area if bsm_finish == True else 11.25 * bsm_area
                elif n_storeys == 2:
                    bld_value = 122.75 * bld_area + 21.05 * bsm_area if bsm_finish == True else 7.30 * bsm_area
                elif n_storeys >= 3:
                    bld_value = 127.94 * bld_area + 16.65 * bsm_area if bsm_finish == True else 5.80 * bsm_area
                    
            elif (const_class == 'Custom'):
                if n_storeys == 1:
                    bld_value = 159.51 * bld_area + 53.65 * bsm_area if bsm_finish == True else 21.65 * bsm_area
                elif n_storeys == 2:
                    bld_value = 163.95 * bld_area + 30.90 * bsm_area if bsm_finish == True else 12.90 * bsm_area
                elif n_storeys >= 3:
                    bld_value = 168.69 * bld_area + 22.55 * bsm_area if bsm_finish == True else 9.60 * bsm_area
                    
            elif (const_class == 'Luxury'):
                if n_storeys == 1:
                    bld_value = 188.84 * bld_area + 59.00 * bsm_area if bsm_finish == True else 22.65 * bsm_area
                elif n_storeys == 2:
                    bld_value = 194.94 * bld_area + 34.55 * bsm_area if bsm_finish == True else 13.85 * bsm_area
                elif n_storeys >= 3:
                    bld_value = 201.09 * bld_area + 25.50 * bsm_area if bsm_finish == True else 10.40 * bsm_area

        # Adapted from Building Portfolio Manual Table 6.2
        elif ('RES3' in bld_occ):
            if ('A' in bld_occ):
                bld_value = 124.25 * bld_area
            elif ('B' in bld_occ):
                bld_value = 109.66 * bld_area  
            elif ('C' in bld_occ):
                bld_value = 201.33 * bld_area
            elif ('D' in bld_occ):
                bld_value = 187.75 * bld_area
            elif ('E' in bld_occ):
                bld_value = 188.48 * bld_area
            elif ('F' in bld_occ):
                bld_value = 174.53 * bld_area
        else:
            print('I do not recognize building type ', bld_occ)
            
        v[i] = bld_value

    return v

# Generate synthetic data

In [35]:
# Building age
# Note: the probability of a building having been built within a time period, e.g., between 1960 and 1970,
#       is usually available from Census data. Here, I am providing a placeholder example only.
s = [1940,1950,1960] # states
p = [1/3,1/3,1/3] # probability of being in each state
n = len(df_Alameda.index) # number of buildings
df_Alameda['YearBuilt'] = setYearBuiltFromDistribution(s,p,n)

# Building code as per California building code implementation dates
df_Alameda['BuildingCode'] = df_Alameda['YearBuilt'].apply(lambda x: f_code(int(x)))

# Occupancy
df_Alameda['Occupancy'] = df_Alameda['Units'].apply(lambda x: 'RES1' if x == 1 else ('RES3A' if x == 2 else 'RES3B'))

# Number of stories
df_Alameda['Stories'] = setStoriesFromHazus('West',df_Alameda)

# Basement
df_Alameda['Basement'] = setBasementFromHazus('West',len(df_Alameda.index))

# Area
df_Alameda['Area'] = setAreaFromHazus('West',df_Alameda)

# Construction class
df_Alameda['ConstructionClass'] = setConstructionClassFromHazus(df_Alameda)

# Replacement cost
df_Alameda['ReplacementCost'] = setReplacementCostFromHazus(df_Alameda)

# Save file
ids = list(range(1,nblds+1))
df_Alameda.drop(columns=['index'],inplace=True)
df_Alameda.insert(0,'index',ids)
df_Alameda.to_csv(data_dir + 'Alameda_Tax_Assessor_Selected.txt', index=False)

## Plot map

In [36]:
# You can change the 'Color' and 'Size' properties to change your plot
# And plot a map with the data underneath
df_plt = df_Alameda.copy()
gdf_plt = gpd.GeoDataFrame(df_plt, geometry=gpd.points_from_xy(df_plt.Longitude, df_plt.Latitude), crs='epsg:4326')

fig = px.scatter_mapbox(gdf_plt, 
                        lat = gdf_plt['Latitude'],
                        lon = gdf_plt['Longitude'],
                        color="ReplacementCost",
                        #size='Area',
                        center={"lat": 37.760, "lon": -122.264},
                        mapbox_style="carto-positron",
                        color_continuous_scale='Viridis',
                        #hover_data = ['Units','Tenure'],
                        zoom=12.2,
                        height=700,
                        width=800
                      )
fig.show()